In [7]:
#ELM tanh
import pandas as pd
import math 
import numpy as np
from scipy.io import loadmat

def set(y):
    for i in range(len(y)):
        if(y[i]>0.5):
            y[i] = 1.0
        if(y[i]<=0.5):
            y[i] = 0.0
    return y

data = loadmat('data5.mat')
data = data['x']
np.random.shuffle(data)

y = data[:,-1]
x = data[:,:-1]
x = (x - np.mean(x,axis=0))/np.std(x,axis=0)

train_size = int(0.35 * len(x))
val_limit = int(0.45*len(x))
test_limit = int(0.65*len(x))
x_tr = x[:train_size]
x_val = x[train_size:val_limit]
x_ts = x[val_limit:test_limit]
y_tr = y[:train_size]
y_val = y[train_size:val_limit]
y_ts = y[val_limit:test_limit]
np.append(x_tr, x[test_limit:])
np.append(y_tr, y[test_limit:])

for p in range (250,1001,50):

        randommat = np.random.randn(x_tr.shape[1]+1,p)
        H = np.append(np.ones((x_tr.shape[0],1)), x_tr, axis=1)
        H = np.dot(H,randommat)
        H = np.tanh(H)
        H = np.matrix(H)

        y_tr = np.matrix(y_tr)
        W = np.dot(H.I,y_tr.T)

        H_val = np.append(np.ones((x_val.shape[0],1)), x_val, axis=1)
        H_val = np.dot(H_val,randommat)
        H_val = np.tanh(H_val)
        H_val = np.matrix(H_val)
        y_pred = np.dot(H_val,W)
        y_predval = np.zeros(len(y_pred))
        for i in range(y_pred.shape[0]):
          y_predval[i] = y_pred[i][0]

        H_ts = np.append(np.ones((x_ts.shape[0],1)), x_ts, axis=1)
        H_ts = np.dot(H_ts,randommat)
        H_ts = np.tanh(H_ts)
        H_ts = np.matrix(H_ts)
        y_predi = np.dot(H_ts,W)
        y_pred = np.zeros(len(y_predi))
        for i in range(y_predi.shape[0]):
          y_pred[i] = y_predi[i][0]

        ypval = set(y_predval)
        yp = set(y_pred)

        y_actual = pd.Series(y_val, name='Actual')
        y_pred = pd.Series(ypval, name='Predicted')
        confmata = pd.crosstab(y_val, y_predval)
        y_actual = pd.Series(y_ts, name='Actual')
        y_pred = pd.Series(yp, name='Predicted')
        confmatb = pd.crosstab(y_actual, y_pred)

        print('\n\nResult for hidden neurons : ' + str(p) + ' :')
        print('Validn:')
        print(confmata)
        print('Test:')
        print(confmatb)
        confmat = np.asarray(confmata)
        tp = confmat[1][1]
        tn = confmat[0][0]
        fp = confmat[0][1]
        fn = confmat[1][0]
        confmat = np.asarray(confmatb)
        tp1 = confmat[1][1]
        tn1 = confmat[0][0]
        fp1 = confmat[0][1]
        fn1 = confmat[1][0]

        Acc = float(tp+tn)/float(tp+tn+fp+fn)
        SE = float(tp)/float(tp+fn)
        SP = float(tn)/float(tn+fp)
        Acc1 = float(tp1+tn1)/float(tp1+tn1+fp1+fn1)
        SE1 = float(tp1)/float(tp1+fn1)
        SP1 = float(tn1)/float(tn1+fp1)

        print('Validn Accuracy : ' + str(Acc))
        print('Validn sensitivity : ' + str(SE))
        print('Validn specificity : ' + str(SP))
        print('Test Accuracy : ' + str(Acc1))
        print('Test sensitivity : ' + str(SE1))
        print('Test specificity : ' + str(SP1))




Result for hidden neurons : 250 :
Validn:
col_0  0.0  1.0
row_0          
0.0     96   18
1.0     22   79
Test:
Predicted  0.0  1.0
Actual             
0.0        181   34
1.0         46  169
Validn Accuracy : 0.813953488372093
Validn sensitivity : 0.7821782178217822
Validn specificity : 0.8421052631578947
Test Accuracy : 0.813953488372093
Test sensitivity : 0.786046511627907
Test specificity : 0.8418604651162791


Result for hidden neurons : 300 :
Validn:
col_0  0.0  1.0
row_0          
0.0     89   25
1.0     22   79
Test:
Predicted  0.0  1.0
Actual             
0.0        176   39
1.0         41  174
Validn Accuracy : 0.7813953488372093
Validn sensitivity : 0.7821782178217822
Validn specificity : 0.7807017543859649
Test Accuracy : 0.813953488372093
Test sensitivity : 0.8093023255813954
Test specificity : 0.8186046511627907


Result for hidden neurons : 350 :
Validn:
col_0  0.0  1.0
row_0          
0.0     92   22
1.0     15   86
Test:
Predicted  0.0  1.0
Actual             
0.0   